In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import nltk,string
import os
import glob
import multiprocessing
import gensim
import numpy as np
import nltk
import re
from random import shuffle

# download the punkt tokenizer
nltk.download('punkt')
print("The punkt tokenizer is downloaded")

The punkt tokenizer is downloaded


[nltk_data] Downloading package punkt to /Users/shelly/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# load review data
file_path = '/Users/shelly/Google Drive/BIA660/final/Web_Scraping/New_20191128/New/*.csv'
addrs = glob.glob(file_path)
# load data from Newly scraped results
for idx, addr in enumerate(addrs):
    if idx == 0:
        review_df = pd.read_csv(addr)
    else:
        new_df = pd.read_csv(addr)
        review_df = pd.concat([review_df, new_df], ignore_index=True)

In [ ]:
review_df.shape

(5243, 7)

In [ ]:
# load data from previous clusters
dir_list = ['/Users/shelly/Google Drive/BIA660/final/Reviews/Cluster0',
            '/Users/shelly/Google Drive/BIA660/final/Reviews/Cluster1-1',
            '/Users/shelly/Google Drive/BIA660/final/Reviews/Cluster2']

for folder in dir_list:
    addrs = glob.glob(folder+'/*.csv')
    for addr in addrs:
        new_df = pd.read_csv(addr).head(50)
        review_df = pd.concat([review_df, new_df], ignore_index=True)
del review_df['Unnamed: 0']

/Users/shelly/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.


In [ ]:
review_df.shape

(6143, 7)

In [ ]:
# load cluster results
cluster_df = pd.read_excel('/Users/shelly/Google Drive/BIA660/final/Cluster_Result.xlsx')

In [ ]:
df = pd.merge(review_df, cluster_df, left_on='book_title', right_on='title')
# replace nan in the review field with ''
df['review'] = df['review'].replace(np.nan, '', regex=True)

In [ ]:
# save it as a csv
df.to_csv('review_join_cluster_6000.csv')

In [ ]:
# df.to_csv('review_join_cluster.csv')

In [ ]:
df.groupby(['cluster']).count()

,book_title,format,helpful,review,sentiment,star_x,title_x,index,No.,book_link,...,star_y,review_amount,kindle_price,paperback_price,hardcover_price,intro,book_rank,product_desc,abt_author,key_words
cluster,,,,,,,,,,,,,,,,,,,,,
0,1615,1536,1219,1615,1615,1615,1615,1615,1615,1615,...,1615,1615,1453,1235,442,1585,1615,1019,548,1615
1,2140,2114,1694,2140,2140,2124,2124,2140,2140,2140,...,2140,2140,2080,1840,1397,2140,1919,1359,441,2140
2,2298,2274,1542,2298,2298,2298,2297,2298,2298,2298,...,2298,2298,2088,1415,1682,2298,1785,1735,147,2298


In [ ]:
# import natural language toolkit
import nltk
# download the punkt tokenizer
nltk.download('punkt')
print("The punkt tokenizer is downloaded")

[nltk_data] Downloading package punkt to /Users/shelly/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

The punkt tokenizer is downloaded


In [ ]:
list(df[df['cluster']==1]['review'])[1241]

'Received a 20 year old book. Not the book pictured'

In [ ]:
cluster = 1
raw_corpus = u''.join(df[df['cluster']==1]['review']+" ")
print("Raw Corpus contains {0:,} characters".format(len(raw_corpus)))

Raw Corpus contains 1,627,166 characters


In [ ]:
# Load the punkt tokenizer
tokenizer = nltk.data.load("tokenizers/punkt/english.pickle")
print("The punkt tokenizer is loaded")
# we tokenize the raw string into raw sentences
raw_sentences = tokenizer.tokenize(raw_corpus)
print("We have {0:,} raw sentences".format(len(raw_sentences)))

The punkt tokenizer is loaded
We have 18,017 raw sentences


In [ ]:
# Clean and split sentence into words
def clean_and_split_str(string):
    strip_special_chars = re.compile("[^A-Za-z]+")
    string = re.sub(strip_special_chars, " ", string)
    return string.strip().split()

In [ ]:
# clean each raw sentences and build the list of sentences
sentences = []
for raw_sent in raw_sentences:
    if len(raw_sent) > 0:
        sentences.append(clean_and_split_str(raw_sent))
print("We have {0:,} clean sentences".format(len(sentences)))

We have 18,017 clean sentences


In [ ]:
print(raw_sentences[10])
print()
print(sentences[10])

While the compliments are wonderful on my new appearance and it's exciting to be 2 sizes smaller, the real changes (I hoped) were happening on the inside.

['While', 'the', 'compliments', 'are', 'wonderful', 'on', 'my', 'new', 'appearance', 'and', 'it', 's', 'exciting', 'to', 'be', 'sizes', 'smaller', 'the', 'real', 'changes', 'I', 'hoped', 'were', 'happening', 'on', 'the', 'inside']


In [ ]:
token_count = sum([len(sentence) for sentence in sentences])
print("The dataset corpus contains {0:,} tokens".format(token_count))

The dataset corpus contains 300,598 tokens


In [ ]:
word2vec_model = gensim.models.word2vec.Word2Vec(
    sg=1,
    seed=1,
    #Seed for the RNG, to make the result reproducible
    workers=multiprocessing.cpu_count(),
    #Number of threads to run in parallel
    size=300,
    #Dimensionality of the resulting word vectors
    min_count=3,
    #Minimum word count threshold
    window=7)
    #Context window length


In [ ]:
word2vec_model.build_vocab(sentences=sentences)
print("The vocabulary is built")
# print("Word2Vec vocabulary length: ", len(list(word2vec_model.vocabulary)))

The vocabulary is built


In [ ]:

#Start training the model
for epoch in range(30):
    # shuffle the documents in each epoch
    shuffle(sentences)
word2vec_model.train(sentences=sentences,
                     total_examples=word2vec_model.corpus_count,
                     epochs=1)
print("Training finished")

(215889, 300598)

Training finished


In [ ]:
#Save the model
word2vec_model.save("result2/word2vec_model_trained_on_cluster0.w2v")
print("Model saved")

Model saved


In [ ]:
def my_word_2_vec(cluster):
    # Convert all the review text into a long string and print its length
    raw_corpus = u''.join(df[df['cluster']==cluster]['review']+" ")
    print("Raw Corpus contains {0:,} characters".format(len(raw_corpus)))
    # Load the punkt tokenizer
    tokenizer = nltk.data.load("tokenizers/punkt/english.pickle")
    print("The punkt tokenizer is loaded")
    # we tokenize the raw string into raw sentences
    raw_sentences = tokenizer.tokenize(raw_corpus)
    print("We have {0:,} raw sentences".format(len(raw_sentences)))
    
    # Clean and split sentence into words
    def clean_and_split_str(string):
        strip_special_chars = re.compile("[^A-Za-z]+")
        string = re.sub(strip_special_chars, " ", string)
        return string.strip().split()
    # clean each raw sentences and build the list of sentences
    sentences = []
    for raw_sent in raw_sentences:
        if len(raw_sent) > 0:
            sentences.append(clean_and_split_str(raw_sent))
    print("We have {0:,} clean sentences".format(len(sentences)))
    
    token_count = sum([len(sentence) for sentence in sentences])
    print("The dataset corpus contains {0:,} tokens".format(token_count))
    
    word2vec_model = gensim.models.word2vec.Word2Vec(
                                                    sg=1,
                                                    seed=1,
                                                    #Seed for the RNG, to make the result reproducible
                                                    workers=multiprocessing.cpu_count(),
                                                    #Number of threads to run in parallel
                                                    size=300,
                                                    #Dimensionality of the resulting word vectors
                                                    min_count=3,
                                                    #Minimum word count threshold
                                                    window=7)
                                                    #Context window length
    word2vec_model.build_vocab(sentences=sentences)
    print("The vocabulary is built")
    for epoch in range(30):
    # shuffle the documents in each epoch
        shuffle(sentences)
        word2vec_model.train(sentences=sentences,
                             total_examples=word2vec_model.corpus_count,
                             epochs=1)
    print("Training finished")
    word2vec_model.save("./results/word2vec_model_trained_on_cluster"+str(cluster)+".w2v")
    print("Model saved")
    return

In [ ]:
my_word_2_vec(0)

Raw Corpus contains 965,846 characters
The punkt tokenizer is loaded
We have 11,229 raw sentences
We have 11,229 clean sentences
The dataset corpus contains 179,511 tokens
The vocabulary is built
Training finished
Model saved


In [ ]:
my_word_2_vec(1)

Raw Corpus contains 1,627,166 characters
The punkt tokenizer is loaded
We have 18,017 raw sentences
We have 18,017 clean sentences
The dataset corpus contains 300,598 tokens
The vocabulary is built
Training finished
Model saved


In [ ]:
my_word_2_vec(2)

Raw Corpus contains 1,465,886 characters
The punkt tokenizer is loaded
We have 16,172 raw sentences
We have 16,172 clean sentences
The dataset corpus contains 270,775 tokens
The vocabulary is built
Training finished
Model saved
